# Build an Agentic RAG Service

Setup an agent service that can interact with a tool service (containing RAG toolsover annual reports).

In [11]:
import nest_asyncio

nest_asyncio.apply()

## Load Data

In [7]:
!mkdir -p 'data/10k/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf' -O 'data/10k/lyft_2021.pdf'

--2024-06-21 00:09:20--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8003::154, 2606:50c0:8002::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1880483 (1.8M) [application/octet-stream]
Saving to: ‘data/10k/uber_2021.pdf’

data/10k/uber_2021. 100%[===================>]   1.79M  --.-KB/s    in 0.07s   

2024-06-21 00:09:21 (24.3 MB/s) - ‘data/10k/uber_2021.pdf’ saved [1880483/1880483]

--2024-06-21 00:09:21--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8003::154, 2606:50c0:8002::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|26

In [1]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [2]:
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/lyft"
    )
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/uber"
    )
    uber_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

In [3]:
if not index_loaded:
    # load data
    lyft_docs = SimpleDirectoryReader(
        input_files=["./data/10k/lyft_2021.pdf"]
    ).load_data()
    uber_docs = SimpleDirectoryReader(
        input_files=["./data/10k/uber_2021.pdf"]
    ).load_data()

    # build index
    lyft_index = VectorStoreIndex.from_documents(lyft_docs)
    uber_index = VectorStoreIndex.from_documents(uber_docs)

    # persist index
    lyft_index.storage_context.persist(persist_dir="./storage/lyft")
    uber_index.storage_context.persist(persist_dir="./storage/uber")

In [4]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

In [5]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description=(
                "Provides information about Lyft financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description=(
                "Provides information about Uber financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

## Setup Agents

Now that we've defined the query tools, we can wrap these under a `ToolService`.

In [27]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-4o")

In [28]:
from agentfile.launchers.local import LocalLauncher
from agentfile.services import AgentService, ToolService
from agentfile.tools import MetaServiceTool
from agentfile.control_plane.fastapi import FastAPIControlPlane
from agentfile.message_queues.simple import SimpleMessageQueue
from agentfile.orchestrators.agent import AgentOrchestrator

from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.tools import FunctionTool
from llama_index.llms.openai import OpenAI


# create our multi-agent framework components
message_queue = SimpleMessageQueue()
control_plane = FastAPIControlPlane(
    message_queue=message_queue,
    orchestrator=AgentOrchestrator(llm=llm),
)

## Define Tool Service
tool_service = ToolService(
    message_queue=message_queue,
    tools=query_engine_tools,
    running=True,
    step_interval=0.5,
)
# define tools here
meta_tools = [
    await MetaServiceTool.from_tool_service(
        t.metadata.name,
        message_queue=message_queue,
        tool_service=tool_service,
    ) for t in query_engine_tools
]


## Define Agent
worker1 = FunctionCallingAgentWorker.from_tools(
    meta_tools,
    llm=OpenAI(),
)
agent1 = worker1.as_agent()
agent_server_1 = AgentService(
    agent=agent1,
    message_queue=message_queue,
    description="Used to answer questions over Uber and Lyft 10K documents",
    service_name="uber_lyft_10k_analyst_agent",
)

## Launch agent 

In [29]:
## Define Launcher
launcher = LocalLauncher(
    [agent_server_1, tool_service],
    control_plane,
    message_queue,
)

In [30]:
# query_str = "What was Lyft's revenue growth in 2021?"
# gets stuck in a loop, should mostly be called once 
query_str = "What are the risk factors for Uber?"
launcher.launch_single(query_str)


INFO:agentfile.message_queues.simple:Consumer dfd6710b-72a7-4170-8478-d232a3d5ee22 has been registered.
INFO:agentfile.message_queues.simple:Consumer 351163e8-dbde-40ca-b6df-34d46c86f12c has been registered.
INFO:agentfile.message_queues.simple:Consumer 402c1095-7340-4a1f-9bde-2d319081d18c has been registered.
INFO:agentfile.message_queues.simple:Consumer 3b9ddaa4-bd17-43c8-8641-999fa65d5a24 has been registered.
INFO:agentfile.message_queues.base:Publishing message: id_='3553caf6-74bc-4fae-9a65-e48082c2a67e' publisher_id='LocalLauncher-6d3105dc-b59f-4340-b185-07868409d873' data={'input': 'What are the risk factors for Uber?', 'task_id': '537d712c-3b30-4a1a-9599-42bc07a64401', 'state': {}, 'agent_id': None} action=<ActionTypes.NEW_TASK: 'new_task'> stats=QueueMessageStats(publish_time=None, process_start_time=None, process_end_time=None) type='control_plane'
INFO:agentfile.services.agent:uber_lyft_10k_analyst_agent launch_local
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/cha


Shutting down.


SystemExit: 0